In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import re
import string

data = DatasetDict()

data = load_dataset("wnkh/MultiMed", 'Chinese')
source_language = 'Chinese'
target_language = 'German'

In [ ]:
def normalize_text_to_chars(text):
    punctuation_to_remove_regex = f"[{re.escape(string.punctuation)}]"
    # Replace punctuation with a space
    text = re.sub(punctuation_to_remove_regex, " ", text).strip() if isinstance(text, str) else text
    # Make each character a word by inserting spaces
    return " ".join(text) if isinstance(text, str) else text

def map_to_translation_format(example):
    """
    Map the dataset fields to a translation format for fine-tuning.
    """
    # Prepare a simple instruction indicating it's a translation task
    instruction = f"Translate the following sentence from {source_language} to {target_language}:"

    # Extract the input sentence and its translation
    input_sentence = example['text']  # Replace 'text' with your dataset's source sentence field
    translation = example[target_language]  # Replace with your dataset's target translation field

    for idx, text in enumerate(translation):
        if text == None:
            input_sentence[idx] = ''
            translation[idx] = ''
    # Return a dictionary with the format for translation
    return {
        "instruction": instruction,
        "input": input_sentence,
        "output": translation
    }

if target_language == 'Chinese':
    data['train'] = data['train'].map(
        lambda example: {target_language: normalize_text_to_chars(example[target_language])}
    )
    data['eval'] = data['eval'].map(
        lambda example: {target_language: normalize_text_to_chars(example[target_language])}
    )
data_train = data["train"].map(map_to_translation_format)
data_eval = data["eval"].map(map_to_translation_format)


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    
    # Format each translation example
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Construct the text input for the model (include the EOS token at the end)
        text = alpaca_prompt.format(instruction, input, output)
        texts.append(text)
    
    return {"text": texts}

# Apply the formatting function to the training set
dataset = data_train.map(formatting_prompts_func, batched=True)

# Similarly, apply the formatting function to the evaluation set
data_eval = data_eval.map(formatting_prompts_func, batched=True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 1,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 300,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = f"mistral_{source_language}_{target_language}",
        report_to = "wandb", # Use this for WandB etc
    ),
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()